In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
number_of_shots = 3

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = "data/output/2023.11.28-4_Llama2_13B_API_3-few-shot_SEMANTIC_SIMILARITY/"

In [9]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [10]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [11]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

In [12]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [13]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [14]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [15]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [16]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [17]:
#data.iloc[181:183]

In [18]:
# data_sel = data #.sample(2)
data_sel = data #.iloc[54:] #.sample(2)

In [19]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [31]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  #print("\n\n##################################################")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_7B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 11.2---

The text does not mention any specific SDG, but it does mention a subway derailment in Brooklyn, which could potentially be related to SDG 11.2, which focuses on "sustainable transportation systems and infrastructure."
1 6099 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text does not directly relate to any specific SDG, but it does mention a celebrity's photo shoot in the Maldives, which is a popular tourist destination. SDG 16 focuses on peace, justice, and strong institutions, and the text does not seem to have any direct impact on these areas.
2 6471 0   ---SDG 16---
3 1786 0   ---SDG 0---
4 2143 0   Sure, I'd be happy 

19 6073 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text does not mention any specific SDGs, but it does relate to the theme of sports and entertainment, which is not directly covered by any of the SDGs. Therefore, the best label for this text is SDG 16, which is focused on peace, justice, and strong institutions.
20 1376 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 11---

The text discusses the impact of a heat dome on millions of people, with specific mentions of Texas, Alabama, and Florida. The focus is on the effects of the heat wave on the population, including the potential for heat-related illnesses and the need for precautions. The SDG that best represents this content is SDG 11, which is focused on sustainable cities and communities.
21 3889 0   Sure! Here's the answer with the la

36 2298 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 3---

The text describes a corrective action plan submitted by a medical facility to address deficiencies found in its pediatric intensive care unit, and the facility's efforts to work with the Department of Health Care Services to finalize a corrective action plan that adequately addresses all of the deficiencies. This text best represents SDG 3, which is "Good Health and Well-being."
37 4226 0   Based on the given text, I would classify it as follows:

---SDG 13---

The text discusses the departure of MAMAMOO's Hwasa from her former agency RBW and her potential signing with PSY's agency P NATION, which could be seen as a positive step towards achieving SDG 13 (Climate Action) as P NATION is known for empowering independent Korean artists and letting them work according to their desires. A

55 2055 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 12---

The text describes the discovery of new cell types and hidden molecular features in the cochlea, which is a part of the inner ear responsible for hearing. This research could potentially lead to new insights and innovations in the field of hearing and hearing loss, which is a key aspect of SDG 12, which aims to ensure sustainable consumption and production patterns.
56 2463 0   Sure! Here's the text you provided, with my label for the SDG(s) it represents:

---SDG 0---

The text does not mention any specific SDGs, so I have labeled it as "SDG 0".
57 193 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 9---

The text is primarily focused o

73 2631 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 1---

The text discusses the concept of poverty and the challenges of measuring it, specifically highlighting the issue of chronic poverty and the use of severe poverty as a proxy for measuring chronic poverty. The text also mentions the differences in poverty rates across countries and the limitations of comparing poverty rates between countries.

Therefore, the most appropriate SDG label for the given text is SDG 1, which is "No Poverty".
74 1250 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 10---

The text discusses wage differentials between different age/skill/contract groups and the impact of reforms on employment levels, which is related to 

...

950 518 15   Sure, I'd be happy to help! Here's the text you provided, with the SDG label based on its content:

---SDG 15---

The text discusses the status of various species in Morocco, including their population sizes and the threats they face. It also mentions the importance of protecting biodiversity and the need to address the uncertainty surrounding the status of many species. Based on this content, the most appropriate SDG label is SDG 15, which focuses on life on land.
951 2066 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 15---

The text discusses the importance of ecotourism as a sector with green growth potential and the need to explore various instruments to capture the international public good benefits provided by protected areas. This aligns with SDG 15, which focuses on life on land and aims to protect, restore, and promote the sus

969 3839 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the use of images and emotions in human rights advocacy, specifically the negative effects of bombarding the public with graphic depictions of suffering and the importance of understanding which emotions motivate individuals to engage in global civic action. This is closely related to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice for all, and effective, accountable, and transparent institutions.
970 3501 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text you provided focuses on the issue of human rights in the context of immigration law enforcement and comprehensive immigration reform, specifically in the U.S.-Mex

983 543 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text you provided discusses the use of torture by military and intelligence personnel of the United States, and the legal and moral implications of such actions. The text also references the Fifth Amendment of the US Constitution and the importance of protecting human rights.

Based on the content of the text, the most appropriate SDG label is SDG 16, which is focused on "Peace, Justice and Strong Institutions." This SDG aims to promote peaceful and inclusive societies, access to justice for all, and effective and accountable institutions at all levels. The text you provided highlights the importance of upholding human rights and the rule of law, which are key components of SDG 16.
984 4331 16   Sure, I'd be happy to help! Here's the classification of the text you provided ba

999 4375 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text you provided discusses the impact of patents on access to science and its benefits, which is a universal and fundamental human right enshrined in international law. The focus on human rights and access to benefits aligns with SDG 16, which is to promote just, peaceful, and inclusive societies.
1000 2522 16   Sure, I'd be happy to help! Here's the classification of the text based on the Sustainable Development Goals (SDGs):

---SDG 5---

The text discusses the experience of women involved in a non-government organization-funded women's empowerment project in Phnom Penh, Cambodia. It explores how power dynamics and social structures in a post-conflict setting affect the outcome of women's self-help groups, highlighting the challenges of assumptions about community and wom

1014 2723 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
1015 4695 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text discusses the struggle for democratization in Nigeria, specifically highlighting the role of human rights NGOs and labor unions in the process. The text also touches on issues of accountability, participation, and egalitarianism within civil society, which are all key components of SDG 16 (peaceful and inclusive societies).
1016 6628 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:

---SDG 16---

The text discusses the creation of global institutions to address public good issues, which falls under the category of "Peace, Justice and Strong Institution

In [32]:
# import time

# cnt = 0
# # cnt = 181

# results = []
# for index, row in data_sel.iterrows():
    
#     text_id = row['text_id']
#     text = row["text"]
#     sdg_true = row["sdg"]
    
#     query = template_part1 + text + template_part2
    
#     body = {
#       "model": "meta-llama/Llama-2-13b-chat-hf",
#       "messages": [
#           {"role": "system", "content": "You are a helpful assistant."}, 
#           {"role": "user", "content": query}],
#       "temperature": 0.0
#     }
    
        
#     try:
#         result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
#         print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
#     except Exception as e:
#         print("#############################################################################")
#         print(e)
#         print("#############################################################################")
        
    
#     result_extended = (cnt, text_id, sdg_true, result)
    
#     with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
#         pickle.dump(result_extended, f)
    
#     results.append(result_extended)
#     cnt+=1
#     time.sleep(0.5)

In [33]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [34]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(289, 1960, 4, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:\n\n---SDG 4---\n\nThe text discusses the issue of quality in education, specifically the decrease in the percentage of qualified teachers, and the impact of Free Primary Education on access to education. These topics are all related to SDG 4, which aims to ensure inclusive and equitable quality education and promote lifelong learning for all.")
(783, 2676, 13, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015:\n\n---SDG 13---\n\nThe text discusses the importance of building trust in climate finance commitments and the challenges in estimating mobilized climate finance, which are both relevant to SDG 13: Climate Action.")
(246, 3956, 4, "  Sure, I'd be happy to hel

In [35]:
len(tuple_list)

1019

In [36]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [37]:
results_processed

,cnt,id,true_label,predicted_label
0,289,1960,4,"Sure, I'd be happy to help! Here's the class..."
1,783,2676,13,"Sure, I'd be happy to help! Here's the class..."
2,246,3956,4,"Sure, I'd be happy to help! Here's the class..."
3,50,2541,0,"Sure, I'd be happy to help! Here's the class..."
4,288,4205,4,"Sure, I'd be happy to help! Here's the text ..."
...,...,...,...,...
1014,691,5312,11,"Sure, I'd be happy to help! Here's the class..."
1015,137,2747,2,"Sure, I'd be happy to help! Here's the class..."
1016,513,4489,8,"Sure, I'd be happy to help! Here's the class..."
1017,723,2110,12,"Sure, I'd be happy to help! Here's the text ..."


In [38]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 0---
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class..."
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class..."
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class..."


In [39]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class...",16
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 0---,0
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",7
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class...",16
1015,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class...",16
1016,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 16---,16
1017,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class...",16


In [40]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,21,0,21,21,0,0,0,21,21,21,21
1,80,79,80,80,79,79,79,80,80,80,80
10,48,47,48,48,47,47,47,48,48,48,48
11,66,59,66,66,59,59,59,66,66,66,66
12,70,68,70,70,68,68,68,70,70,70,70
13,62,61,62,62,61,61,61,62,62,62,62
14,59,59,59,59,59,59,59,59,59,59,59
15,64,62,64,64,62,62,62,64,64,64,64
16,66,60,66,66,60,60,60,66,66,66,66


In [41]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [42]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       1.00      0.36      0.53        59
           1       0.70      0.93      0.80        60
           2       0.92      0.80      0.86        60
           3       0.78      0.90      0.84        60
           4       0.82      0.92      0.87        60
           5       0.86      0.92      0.89        60
           6       0.98      0.92      0.95        60
           7       0.89      0.85      0.87        60
           8       0.66      0.65      0.66        60
           9       0.84      0.72      0.77        60
          10       0.79      0.63      0.70        60
          11       0.79      0.87      0.83        60
          12       0.80      0.93      0.86        60
          13       0.89      0.92      0.90        60
          14       0.97      0.95      0.96        60
          15       0.89      0.95      0.92        60
          16       0.83      0.92      0.87        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
